In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'2.0.1+cu118'

# MNIST数据集手写数字识别

## 1.数据集介绍

MNIST 包括了6万张28x28的训练样本，1万张测试样本，可以说它就是计算机视觉里面的Hello World。最关键的是使用cpu也可以跑，所以我们这里也会使用MNIST来进行实战。

这里我们也自己从头搭建一个卷积神经网络，希望能够达到一个较高的准确率。

## 2.手写数字识别

这里先定义一些超参数，电脑性能不足的就减小批量

In [10]:
# 批量大小
batch_size = 512 
# 总共训练批次
epochs = 20 
# 使用'cup'还是'gpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

在torchvision这个包中就可以用于下载MNIST数据集，所以可以直接去使用。第一次下载需要一些时间下载，当然下载过了就不会再次下载了。

这里我们直接使用DataLoader来对数据进行读取，后面会单独出一期关于DataLoader的知识。

##### 训练集与测试集下载

In [6]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

100%|█████████████████████████████████████████████████████████████████████| 9912422/9912422 [02:45<00:00, 59854.44it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 28881/28881 [00:01<00:00, 17214.38it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|█████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:26<00:00, 62048.34it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



## 3.自定义创建Model

* 要先继承nn.Module且在其构造函数中需调用nn.Module的构造函数
* 不用写反向传播函数，nn.Module能够利用autograd自动实现反向传播
* Module中的可学习参数可以通过named_parameters()或者parameters()返回迭代器

这里我们自定义一个网络，网络包含两个卷积层，分别是conv1和conv2，然后紧接着两个线性层作为输出，最后输出10个维度，这10个维度我们作为0-9的标识来确定识别出的是那个数字。

每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28，所以格式就是这样的 —— [batch,1,28,28]

Conv2d: 第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
Linear：第一个参数指输入通道数，第二个参数指输出通道数

In [7]:
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5) 
        self.conv2 = nn.Conv2d(10, 20, 3) 
        self.fc1 = nn.Linear(20*10*10, 500) 
        self.fc2 = nn.Linear(500, 10) # 10分类
        
    def forward(self,x):
        in_size = x.size(0) # BATCH_SIZE=512，输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，进行自动推算）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

卷积的公式，最好记着：输出大小 = ((N - M + 2P) / S) + 1

输入图像的大小是 N × N，卷积核的大小是 M × M，步幅（stride）为 S，填充（padding）为 P

打印查看模型长什么样，是否正确。

In [8]:
net = MnistNet()
print(net)

MnistNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2000, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


可以打印定义好名字里的权重和偏置项

In [9]:
for name, parameter in net.named_parameters():
    print(name, parameter,parameter.size())

conv1.weight Parameter containing:
tensor([[[[ 0.0443,  0.0222,  0.0170,  0.1916,  0.1348],
          [-0.1929,  0.1519, -0.1807, -0.1214, -0.1014],
          [ 0.1826,  0.0255, -0.0960,  0.0436,  0.1469],
          [-0.1778,  0.0831, -0.0293, -0.0494, -0.1437],
          [ 0.0892,  0.1882, -0.0203, -0.1730,  0.0614]]],


        [[[-0.0362,  0.0380,  0.0790, -0.1535, -0.0465],
          [ 0.0450, -0.1768, -0.0534,  0.0363,  0.0672],
          [-0.0019, -0.0868,  0.1701,  0.1655,  0.0056],
          [ 0.0413, -0.0175,  0.1054, -0.0454,  0.1005],
          [ 0.1700,  0.1073,  0.0084,  0.1025, -0.0304]]],


        [[[-0.0446,  0.0768,  0.1197,  0.0150, -0.0052],
          [ 0.1346, -0.1450, -0.1986, -0.0118, -0.1841],
          [-0.1355, -0.1696,  0.0133,  0.1899,  0.0200],
          [-0.1700, -0.1997,  0.1026,  0.1700, -0.0515],
          [-0.0114, -0.0995,  0.0858,  0.0654,  0.1008]]],


        [[[-0.0822,  0.1187, -0.0238,  0.1033,  0.0478],
          [-0.0369, -0.1912,  0.0380, -0.

实例化后使用.to方法将网络移动到GPU

优化器就选择最有效的Adam

In [13]:
net = net.to(device)
optimizer = optim.Adam(net.parameters())

然后需要定义一个train函数，这里面封装着我的一些训练的操作，损失函数这里用的是负对数似然损失函数，你可以自己替换。

In [16]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('当前Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

测试的操作几乎相同

In [17]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset) * 100. 
    print('\n验证集: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, 
        len(test_loader.dataset), acc))

接下来训练也是一步到位

In [18]:
for epoch in range(1, epochs + 1):
    train(net, device, train_loader, optimizer, epoch)
    test(net, device, test_loader)

当前Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.317121
当前Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.194249
当前Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.175830

验证集: Average loss: 0.1077, Accuracy: 9689/10000 (97%)

当前Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.119607
当前Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.080488
当前Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.075732

验证集: Average loss: 0.0659, Accuracy: 9791/10000 (98%)

当前Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.053178
当前Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.051680
当前Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.033485

验证集: Average loss: 0.0484, Accuracy: 9836/10000 (98%)

当前Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.049852
当前Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.034732
当前Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.055895

验证集: Average loss: 0.0477, Accuracy: 9837/10000 (98%)

当前Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.029753
当前Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.047640
当前Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.016

虽然跑的不多，只有区区20轮，不过准确率也还是相当的高的，最终准确率也是达到了99%

深度学习中的手写数字识别是一个常见的入门项目，它可以帮助初学者熟悉神经网络的基本概念和训练过程。通过实践手写数字识别，可以学习到数据预理、模型构建、训练和评估等关键步骤。

但并不是说的你的模型就特别好，如果你搭建的模型连Minist都搞不定，那你的模型一定不是很好。